In [1]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import transforms
from torchvision.models import vgg11
from torch.utils.data import Dataset, DataLoader,random_split, SubsetRandomSampler
import numpy as np
from sklearn.metrics import f1_score

from dataset import TrainDataset, TestDataset, img_transform, TrainDatasetAgeAugmentation
from avgMeter import AverageMeter

import warnings
warnings.filterwarnings("ignore")

from focal_loss import FocalLoss
from efficientnet_pytorch import EfficientNet
import copy
import random
import os 

torch.manual_seed(0)
torch.cuda.manual_seed(0)
np.random.seed(0)
random.seed(0)



In [2]:
transform = img_transform()


In [3]:
train_root = '/opt/ml/input/data/train/images'
test_root = '/opt/ml/input/data/eval'


In [4]:
batch_size = 128
lr = 0.0001
num_epochs = 60
model_name = 'effnetb4-batchsize_' + str(batch_size) + '-lr_' + str(lr).split('.')[1] + '-epoch_' + str(num_epochs) + '-CenterCrop' + '-scheduler' \
                + '-nofreeze' + '-fix_data' + '-no_overlap' + '-focal_loss'
log_dir = '/opt/ml/code/log/' + model_name + '.txt' 
save_dir = '/opt/ml/code/trained_models/' + model_name + '.pt'

In [5]:
data = TrainDataset(train_root, input_size = 224, transform = transform)

In [6]:
'''마스크 쓴 데이터의 0.6을 validation으로 옮김.'''
# indices = list(range(len(data)))
# split_idx = int(len(data) * 0.8)
# train_idx, valid_idx = [], []
# filter_idx = {0,4}
# for i in range(split_idx):
#     if (i%7) in filter_idx:
#         train_idx.append(i)
#     else:
#         prob = random.uniform(0,1)
#         if prob > 0.4:
#             valid_idx.append(i)
#         else:
#             train_idx.append(i)
# valid_idx.extend(indices[split_idx:])
# print(len(train_idx))
# print(len(valid_idx))


# train_sampler = SubsetRandomSampler(train_idx)
# valid_sampler = SubsetRandomSampler(valid_idx)

# train_loader = DataLoader(data, batch_size=batch_size, num_workers = 4, sampler=train_sampler, pin_memory=True, shuffle=False)
# valid_loader = DataLoader(data, batch_size=batch_size, num_workers = 4, sampler=valid_sampler, pin_memory=True, shuffle=False)

'마스크 쓴 데이터의 0.6을 validation으로 옮김.'

In [7]:
indices = list(range(len(data)))
split_idx = int(len(data) * 0.8)
train_idx, valid_idx = indices[:split_idx], indices[split_idx:]


train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = DataLoader(data, batch_size=batch_size, num_workers = 4, sampler=train_sampler, pin_memory=True, shuffle=False)
valid_loader = DataLoader(data, batch_size=batch_size, num_workers = 4, sampler=valid_sampler, pin_memory=True, shuffle=False)

In [8]:
# train, val = random_split(data, [int(len(data)*0.8), int(len(data)*0.2)])
# train_loader = DataLoader(train, batch_size=batch_size, num_workers = 4,  pin_memory=True, shuffle=True)
# valid_loader = DataLoader(val, batch_size=batch_size, num_workers = 4,  pin_memory=True, shuffle=False)

In [9]:
print(model_name)

effnetb4-batchsize_128-lr_0001-epoch_60-CenterCrop-scheduler-nofreeze-fix_data-no_overlap-focal_loss


In [10]:
print(save_dir)

/opt/ml/code/trained_models/effnetb4-batchsize_128-lr_0001-epoch_60-CenterCrop-scheduler-nofreeze-fix_data-no_overlap-focal_loss.pt


In [11]:
model = EfficientNet.from_pretrained('efficientnet-b4', num_classes=18)

Loaded pretrained weights for efficientnet-b4


In [12]:
# model._fc = nn.Linear(in_features=1280, out_features=18, bias=True)
model.cuda()
print('haha')

haha


In [13]:
# for n, p in model.named_parameters():
#     if '_fc' not in n:
#         p.requires_grad = False

In [14]:
# criterion = nn.CrossEntropyLoss()
criterion = FocalLoss(gamma = 3)
optimizer = Adam(model.parameters(), lr=lr)

scheduler = ReduceLROnPlateau(optimizer, mode = 'min', factor = 0.1, patience = 5)

In [15]:
best_val_acc = 0.0
best_model_wts = copy.deepcopy(model.state_dict())
train_loss, train_acc = AverageMeter(), AverageMeter()
valid_loss, valid_acc = AverageMeter(), AverageMeter()
f1 = AverageMeter()
with open(log_dir, 'w') as log:
    for epoch in range(num_epochs):
        train_loss.reset()
        train_acc.reset()
        for iter, (img, label) in enumerate(train_loader):
            optimizer.zero_grad()

            img, label = img.float().cuda(), label.cuda()

            pred_logit = model(img)

            loss = criterion(pred_logit, label)

            loss.backward()
            optimizer.step()

            pred_label = pred_logit.argmax(-1)
            acc = (pred_label == label).sum().float() / img.size(0)

            train_loss.update(loss.item(), len(img))
            train_acc.update(acc, len(img))
        
            
            
        valid_loss.reset()
        valid_acc.reset()
        f1.reset()
        for img, label in valid_loader:
            img, label = img.float().cuda(), label.cuda()

            with torch.no_grad():
                pred_logit = model(img)


            loss = criterion(pred_logit, label)

            pred_label = pred_logit.argmax(-1)
            acc = (pred_label == label).sum().float() / img.size(0)
            
            valid_loss.update(loss.item(), len(img))
            valid_acc.update(acc, len(img))
            f1.update(f1_score(pred_label.cpu(), label.cpu(), average='weighted'), len(img))
            
        train_loss_val = train_loss.avg
        train_acc_val = train_acc.avg
        valid_loss_val = valid_loss.avg
        valid_acc_val = valid_acc.avg
        f1_val = f1.avg
        
        print("epoch [%3d/%3d] | Train Loss %.4f | Train Acc %.4f | Valid Loss %.4f | Valid Acc %.4f | f1_score %.4f" %
            (epoch+1, num_epochs, train_loss_val, train_acc_val, valid_loss_val, valid_acc_val, f1_val))
        
        if valid_acc_val > best_val_acc:
            best_val_acc = valid_acc_val
            best_model_wts = copy.deepcopy(model.state_dict())
        
        scheduler.step(valid_loss_val)
        # Train Log Writing
        log.write("epoch [%3d/%3d] | Train Loss %.4f | Train Acc %.4f | Valid Loss %.4f | Valid Acc %.4f | f1_score %.4f\n" %
            (epoch+1, num_epochs, train_loss_val, train_acc_val, valid_loss_val, valid_acc_val, f1_val))

epoch [  1/ 60] | Train Loss 0.8229 | Train Acc 0.7083 | Valid Loss 0.2118 | Valid Acc 0.8561 | f1_score 0.8649
epoch [  2/ 60] | Train Loss 0.1106 | Train Acc 0.8989 | Valid Loss 0.1309 | Valid Acc 0.8741 | f1_score 0.8733
epoch [  3/ 60] | Train Loss 0.0505 | Train Acc 0.9319 | Valid Loss 0.1158 | Valid Acc 0.8862 | f1_score 0.8868
epoch [  4/ 60] | Train Loss 0.0334 | Train Acc 0.9508 | Valid Loss 0.1145 | Valid Acc 0.8899 | f1_score 0.8912
epoch [  5/ 60] | Train Loss 0.0197 | Train Acc 0.9687 | Valid Loss 0.1031 | Valid Acc 0.8942 | f1_score 0.8939
epoch [  6/ 60] | Train Loss 0.0146 | Train Acc 0.9774 | Valid Loss 0.1146 | Valid Acc 0.8833 | f1_score 0.8806
epoch [  7/ 60] | Train Loss 0.0108 | Train Acc 0.9835 | Valid Loss 0.1242 | Valid Acc 0.8910 | f1_score 0.8895
epoch [  8/ 60] | Train Loss 0.0082 | Train Acc 0.9873 | Valid Loss 0.1143 | Valid Acc 0.8907 | f1_score 0.8892
epoch [  9/ 60] | Train Loss 0.0093 | Train Acc 0.9878 | Valid Loss 0.1236 | Valid Acc 0.8921 | f1_score

KeyboardInterrupt: 

In [16]:
model.__class__.__name__

'EfficientNet'

In [17]:
model.load_state_dict(best_model_wts)
torch.save(model, save_dir)